In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext line_profiler

In [ ]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import pathlib
from functools import partial

import time
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble']=r"\usepackage{bm}"
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import jax
import jax.numpy as jnp
# jax.config.update("jax_enable_x64", True)
gpus = jax.devices()
jax.config.update("jax_default_device", gpus[0])

import diffrax
import equinox as eqx
import optax

from haiku import PRNGSequence

In [ ]:
import exciting_environments as excenvs

import exciting_exciting_systems
from exciting_exciting_systems.models import NeuralEulerODEPendulum, NeuralODEPendulum
from exciting_exciting_systems.models.model_utils import simulate_ahead_with_env
from exciting_exciting_systems.models.model_training import ModelTrainer
from exciting_exciting_systems.excitation import loss_function, Exciter

from exciting_exciting_systems.utils.density_estimation import (
    update_density_estimate_single_observation, update_density_estimate_multiple_observations, DensityEstimate
)
from exciting_exciting_systems.utils.signals import aprbs
from exciting_exciting_systems.evaluation.plotting_utils import (
    plot_sequence, append_predictions_to_sequence_plot, plot_sequence_and_prediction, plot_model_performance
)
from exciting_exciting_systems.evaluation.experiment_utils import (
    get_experiment_ids, load_experiment_results, quick_eval_pendulum, evaluate_experiment_metrics, evaluate_algorithm_metrics, evaluate_metrics
)

---

In [ ]:
def identity(x):
    return x

def featurize_theta(obs_action):
    """The angle itself is difficult to properly interpret in the loss as angles
    such as 1.99 * pi and 0 are essentially the same. Therefore the angle is 
    transformed to sin(phi) and cos(phi) for comparison in the loss."""

    feat_obs_action = np.stack([np.sin(obs_action[..., 0] * np.pi), np.cos(obs_action[..., 0] * np.pi)], axis=-1)
    feat_obs_action = np.concatenate([feat_obs_action, obs_action[..., 1:]], axis=-1)
    
    return feat_obs_action

## DMPE:

In [ ]:
batch_size = 1
tau = 2e-2

env = excenvs.make(
    env_id='Pendulum-v0',
    batch_size=batch_size,
    action_constraints={"torque": 8},
    static_params={"g": 9.81, "l": 1, "m": 1},
    solver=diffrax.Tsit5(),
    tau=tau,
)

In [ ]:
results_path = pathlib.Path("/home/hvater@uni-paderborn.de/projects/exciting-exciting-systems/eval/results/dmpe/")

In [ ]:
for identifier in get_experiment_ids(results_path)[-4:]:
    quick_eval_pendulum(env, identifier, results_path, NeuralEulerODEPendulum)

In [ ]:
params, observations, actions, model = load_experiment_results(
    exp_id=get_experiment_ids(results_path)[-2],
    results_path=results_path,
    model_class=NeuralEulerODEPendulum
)

In [ ]:
params

In [ ]:
quick_eval_pendulum(env, get_experiment_ids(results_path)[-2], results_path, NeuralEulerODEPendulum)

- why does that one dmpe run diverge?
- What does the model look like for the run?

-> first reproduce! **Check**
- model does predict the behaviour correctly
- the input optimization seems to get stuck in that position
- Why though?

-> Very small gradients in the loss w.r.t. the actions? does not seem to be the case

- the learning rate is to small to move the inputs away from their old values, why is it able to do this for other smaller gradients then?

In [ ]:
results = evaluate_experiment_metrics(observations, actions, featurize=identity)

In [ ]:
results

In [ ]:
get_experiment_ids(results_path)[-3:]

In [ ]:
dmpe_results = evaluate_algorithm_metrics(
    identifiers=get_experiment_ids(results_path)[-3:],
    results_path=results_path,
)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame.from_dict(dmpe_results)

In [ ]:
results = evaluate_metrics(
    algorithm_names=["dmpe", "sgoats", "goats"],
    n_results=8,
    results_parent_path=pathlib.Path("/home/hvater@uni-paderborn.de/projects/exciting-exciting-systems/eval/results/"),
    featurize=None
)
results

In [ ]:
df_dmpe = pd.DataFrame.from_dict(results["dmpe"])
df_goats = pd.DataFrame.from_dict(results["goats"])
df_sgoats = pd.DataFrame.from_dict(results["sgoats"])

df_mean = pd.DataFrame()
df_mean["dmpe"] = df_dmpe.mean(axis=0)
df_mean["goats"] = df_goats.mean(axis=0)
df_mean["sgoats"] = df_sgoats.mean(axis=0)

df_std = pd.DataFrame()
df_std["dmpe"] = df_dmpe.std(axis=0)
df_std["goats"] = df_goats.std(axis=0)
df_std["sgoats"] = df_sgoats.std(axis=0)

In [ ]:
df_mean, df_std

## GOATS + sGOATS:

In [ ]:
algorithm = "sgoats"
results_path = pathlib.Path(f"/home/hvater@uni-paderborn.de/projects/exciting-exciting-systems/eval/results/{algorithm}/")

In [ ]:
identifiers = get_experiment_ids(results_path)
identifiers

In [ ]:
for identifier in get_experiment_ids(results_path):
    quick_eval_pendulum(env, identifier, results_path, None)

## iGOATS: